In [4]:
import pygame
import random
import math

pygame.init()


# Constants
WINDOW_SIZE = (600, 600)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255, 255, 0)
LIGHT_GREEN = (144, 238, 144)  # For path prediction
FONT_COLOR = (0, 0, 0)
FONT_SIZE = 24
step_size = 3
tolerance = 10
ARROW_LENGTH = 20
NUM_GOALS = 3
PATH_POINTS = 10  # Number of points to show in predicted path
MAX_DISTANCE_FROM_HUMAN = 100

NUM_OBSTACLES = 5  # Number of obstacles
OBSTACLE_RADIUS = 20  # Radius of obstacles
COLLISION_BUFFER = 5  # Extra buffer for collision detection
GRAY = (128, 128, 128)  # Color for obstacles

# Add this to your global variables section
obstacles = []

# Increased step size for agent1 (green dot)
agent1_step_size = 3.5  # Doubled from the original step size

# Set up display
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("2D Environment with Path Prediction")

# Load font for rendering text
font = pygame.font.Font(None, FONT_SIZE)

# Agent positions and directions
agent1_pos = [300, 300]
agent2_pos = [300, 300]
agent3_pos = [300, 300]
agent1_dir = [0, 0]
agent2_dir = [0, 0]
agent3_dir = [0, 0]
reached_goal = False

# Create multiple targets
targets = []
for _ in range(NUM_GOALS):
    targets.append([random.randint(0, WINDOW_SIZE[0]), random.randint(0, WINDOW_SIZE[1])])

current_target_idx = 0
prev_human_pos = agent2_pos.copy()



def find_alternative_path(current_pos, target_pos, num_rays=16, ray_length=50):
    """Find alternative direction when direct path is blocked"""
    best_direction = None
    best_score = float('-inf')
    
    # Generate rays in different directions
    for i in range(num_rays):
        angle = (2 * math.pi * i) / num_rays
        dx = math.cos(angle)
        dy = math.sin(angle)
        
        # Check if this direction is blocked
        new_pos = [
            current_pos[0] + dx * ray_length,
            current_pos[1] + dy * ray_length
        ]
        
        if not check_collision(current_pos, new_pos):
            # Score this direction based on:
            # 1. How close it gets us to the target
            # 2. How similar it is to the direct path
            
            # Direct vector to target
            to_target_dx = target_pos[0] - current_pos[0]
            to_target_dy = target_pos[1] - current_pos[1]
            target_dist = math.sqrt(to_target_dx**2 + to_target_dy**2)
            
            # Distance after moving in this direction
            new_target_dx = target_pos[0] - new_pos[0]
            new_target_dy = target_pos[1] - new_pos[1]
            new_dist = math.sqrt(new_target_dx**2 + new_target_dy**2)
            
            # Dot product to measure direction similarity
            if target_dist > 0:  # Normalize only if not at target
                dot_product = (to_target_dx * dx + to_target_dy * dy) / target_dist
            else:
                dot_product = 1
                
            # Combine factors into score
            distance_improvement = target_dist - new_dist
            direction_score = dot_product
            
            score = distance_improvement + direction_score * 50
            
            if score > best_score:
                best_score = score
                best_direction = (dx, dy)
    
    return best_direction



def generate_obstacles():
    """Generate non-overlapping obstacles"""
    obstacles.clear()
    for _ in range(NUM_OBSTACLES):
        while True:
            pos = [random.randint(OBSTACLE_RADIUS, WINDOW_SIZE[0]-OBSTACLE_RADIUS),
                  random.randint(OBSTACLE_RADIUS, WINDOW_SIZE[1]-OBSTACLE_RADIUS)]
            
            # Check if obstacle overlaps with agents' starting positions
            if (distance(pos, agent1_pos) > OBSTACLE_RADIUS * 2 and
                distance(pos, agent2_pos) > OBSTACLE_RADIUS * 2 and
                distance(pos, agent3_pos) > OBSTACLE_RADIUS * 2):
                
                # Check if obstacle overlaps with other obstacles
                overlap = False
                for other_pos in obstacles:
                    if distance(pos, other_pos) < OBSTACLE_RADIUS * 2.5:
                        overlap = True
                        break
                
                if not overlap:
                    obstacles.append(pos)
                    break


def generate_targets():
    """Generate targets that don't overlap with obstacles"""
    targets.clear()
    for _ in range(NUM_GOALS):
        while True:
            pos = [random.randint(0, WINDOW_SIZE[0]), 
                  random.randint(0, WINDOW_SIZE[1])]
            
            # Check if target overlaps with obstacles
            valid_position = True
            for obstacle_pos in obstacles:
                if distance(pos, obstacle_pos) < OBSTACLE_RADIUS * 1.5:
                    valid_position = False
                    break
            
            # Check if target overlaps with other targets
            for target in targets:
                if distance(pos, target) < OBSTACLE_RADIUS:
                    valid_position = False
                    break
            
            if valid_position:
                targets.append(pos)
                break

def line_circle_intersection(start, end, circle_center, radius):
    """Check if a line segment intersects with a circle"""
    # Vector from start to end
    dx = end[0] - start[0]
    dy = end[1] - start[1]
    
    # Vector from start to circle center
    cx = circle_center[0] - start[0]
    cy = circle_center[1] - start[1]
    
    # Length of line segment squared
    l2 = dx*dx + dy*dy
    
    if l2 == 0:
        # Start and end are the same point
        return distance(start, circle_center) <= radius
    
    # Projection of circle center onto line segment
    t = max(0, min(1, (cx*dx + cy*dy) / l2))
    
    # Point on line segment closest to circle center
    projection_x = start[0] + t * dx
    projection_y = start[1] + t * dy
    
    # Check if distance from closest point is less than radius
    return distance([projection_x, projection_y], circle_center) <= radius

def check_collision(pos, new_pos):
    """Check if moving to new_pos would result in a collision with any obstacle"""
    for obstacle_pos in obstacles:
        if line_circle_intersection(pos, new_pos, obstacle_pos, OBSTACLE_RADIUS + COLLISION_BUFFER):
            return True
    return False




# Update calculate_path_points to show the pathfinding prediction
def calculate_path_points(start_pos, target_pos, num_points=PATH_POINTS):
    """Calculate points along the predicted path, accounting for obstacles"""
    points = []
    current_pos = start_pos.copy()
    
    for _ in range(num_points):
        points.append((int(current_pos[0]), int(current_pos[1])))
        
        # Calculate direction to target
        dx = target_pos[0] - current_pos[0]
        dy = target_pos[1] - current_pos[1]
        dist = math.sqrt(dx**2 + dy**2)
        
        if dist < agent1_step_size:
            points.append((int(target_pos[0]), int(target_pos[1])))
            break
            
        # Normalize
        if dist > 0:
            dx /= dist
            dy /= dist
            
        # Check if direct path is blocked
        new_pos = [
            current_pos[0] + dx * agent1_step_size,
            current_pos[1] + dy * agent1_step_size
        ]
        
        if check_collision(current_pos, new_pos):
            # Find alternative direction
            alt_direction = find_alternative_path(current_pos, target_pos)
            if alt_direction:
                dx, dy = alt_direction
            else:
                break
        
        # Move along path
        current_pos[0] += dx * agent1_step_size
        current_pos[1] += dy * agent1_step_size
        
    return points


def draw_predicted_path(surface, points, color):
    """Draw a dotted line showing the predicted path"""
    if len(points) < 2:
        return
        
    # Draw dots with decreasing size and opacity
    for i in range(len(points) - 1):
        # Calculate dot size and opacity
        size = max(1, int(5 * (1 - i/len(points))))
        alpha = int(255 * (1 - i/len(points)))
        
        # Create a surface for the dot with transparency
        dot_surface = pygame.Surface((size*2, size*2), pygame.SRCALPHA)
        color_with_alpha = (*color[:3], alpha)
        pygame.draw.circle(dot_surface, color_with_alpha, (size, size), size)
        
        # Blit the dot
        surface.blit(dot_surface, (points[i][0] - size, points[i][1] - size))

def distance(pos1, pos2):
    return math.sqrt((pos1[0] - pos2[0])**2 + (pos1[1] - pos2[1])**2)

def predict_human_target():
    global current_target_idx, prev_human_pos
    

    human_dx = agent2_pos[0] - prev_human_pos[0]
    human_dy = agent2_pos[1] - prev_human_pos[1]
    
    if human_dx == 0 and human_dy == 0:
        return current_target_idx
    
    best_score = float('-inf')
    best_target_idx = current_target_idx
    
    for i, target in enumerate(targets):
        to_target_dx = target[0] - agent2_pos[0]
        to_target_dy = target[1] - agent2_pos[1]
        
        movement_mag = math.sqrt(human_dx**2 + human_dy**2)
        target_mag = math.sqrt(to_target_dx**2 + to_target_dy**2)
        
        if movement_mag == 0 or target_mag == 0:
            continue
            
        alignment = (human_dx * to_target_dx + human_dy * to_target_dy) / (movement_mag * target_mag)
        
        dist = distance(agent2_pos, target)
        max_dist = math.sqrt(WINDOW_SIZE[0]**2 + WINDOW_SIZE[1]**2)
        distance_factor = 1 - (dist / max_dist)
        
        score = (alignment * 0.7) + (distance_factor * 0.3)
        
        if score > best_score:
            best_score = score
            best_target_idx = i
            
        
    if best_target_idx != current_target_idx:
        agent1_pos[0] = agent2_pos[0]
        agent1_pos[1] = agent2_pos[1]
    
    return best_target_idx



def calculate_gamma():
    max_distance = math.sqrt(WINDOW_SIZE[0]**2 + WINDOW_SIZE[1]**2)
    distance_w = distance(agent1_pos, targets[current_target_idx])
    distance_h = distance(agent2_pos, targets[current_target_idx])
    
    # Calculate normalized distances
    normalized_distance_w = distance_w / max_distance
    normalized_distance_h = distance_h / max_distance
    
    # Calculate distance between human and X
    distance_h_to_x = distance(agent2_pos, agent3_pos)
    
    # Base gamma calculation
    if normalized_distance_w + normalized_distance_h == 0:
        base_gamma = 0.5
    else:
        base_gamma = normalized_distance_h / (normalized_distance_w + normalized_distance_h)
    
    # Adjust gamma based on distance to human
    distance_factor = min(1.0, distance_h_to_x / MAX_DISTANCE_FROM_HUMAN)
    
    # Smoothly reduce gamma as X gets further from human
    adjusted_gamma = base_gamma * (1 - distance_factor)
    
    # Add minimum weight for human position to ensure X stays somewhat close
    final_gamma = min(0.8, adjusted_gamma)  # Cap maximum gamma at 0.8
    
    return final_gamma

def normalize_vector(dx, dy):
    length = math.sqrt(dx**2 + dy**2)
    if length == 0:
        return 0, 0
    return dx/length, dy/length

def draw_arrow(surface, color, start_pos, direction, length=ARROW_LENGTH):
    if direction[0] == 0 and direction[1] == 0:
        return
        
    end_x = start_pos[0] + direction[0] * length
    end_y = start_pos[1] + direction[1] * length
    
    pygame.draw.line(surface, color, start_pos, (end_x, end_y), 2)
    
    arrow_size = 7
    angle = math.atan2(direction[1], direction[0])
    arrow1_x = end_x - arrow_size * math.cos(angle + math.pi/6)
    arrow1_y = end_y - arrow_size * math.sin(angle + math.pi/6)
    arrow2_x = end_x - arrow_size * math.cos(angle - math.pi/6)
    arrow2_y = end_y - arrow_size * math.sin(angle - math.pi/6)
    
    pygame.draw.line(surface, color, (end_x, end_y), (arrow1_x, arrow1_y), 2)
    pygame.draw.line(surface, color, (end_x, end_y), (arrow2_x, arrow2_y), 2)

def move_agent1():
    global agent1_pos, reached_goal, agent1_dir, current_target_idx
    current_target = targets[current_target_idx]
    
    # Calculate direct path
    dx = current_target[0] - agent1_pos[0]
    dy = current_target[1] - agent1_pos[1]
    distance_to_target = math.sqrt(dx**2 + dy**2)
    
    if distance_to_target == 0:
        return
        
    # Normalize direct vector
    direct_dx = dx / distance_to_target
    direct_dy = dy / distance_to_target
    
    # Check if direct path is blocked
    new_pos = [
        agent1_pos[0] + direct_dx * agent1_step_size,
        agent1_pos[1] + direct_dy * agent1_step_size
    ]
    
    if check_collision(agent1_pos, new_pos):
        # Find alternative direction
        alt_direction = find_alternative_path(agent1_pos, current_target)
        if alt_direction:
            direct_dx, direct_dy = alt_direction
    
    # Update direction indicator
    agent1_dir = [direct_dx, direct_dy]
    
    # Try to move in the chosen direction
    new_pos = [
        agent1_pos[0] + direct_dx * agent1_step_size,
        agent1_pos[1] + direct_dy * agent1_step_size
    ]
    
    if not check_collision(agent1_pos, new_pos):
        # Move if no collision
        agent1_pos[0] = new_pos[0]
        agent1_pos[1] = new_pos[1]
        
        # Check if we've reached the target
        current_distance = distance(agent1_pos, current_target)
        if current_distance < agent1_step_size:
            if current_target_idx == len(targets) - 1:
                reached_goal = True
                pygame.time.set_timer(pygame.USEREVENT, 1000)
            else:
                current_target_idx += 1
                agent1_pos[0] = agent2_pos[0]
                agent1_pos[1] = agent2_pos[1]

def move_agent2(dx, dy):
    global agent2_pos, agent2_dir, prev_human_pos
    prev_human_pos = agent2_pos.copy()
    
    new_x = agent2_pos[0] + dx
    new_y = agent2_pos[1] + dy
    
    # Check for collision before moving
    if not check_collision(agent2_pos, [new_x, new_y]):
        agent2_pos[0] = max(0, min(WINDOW_SIZE[0], new_x))
        agent2_pos[1] = max(0, min(WINDOW_SIZE[1], new_y))
        
        if dx != 0 or dy != 0:
            agent2_dir = normalize_vector(dx, dy)

def update_agent3():
    global agent3_pos, agent3_dir
    gamma = calculate_gamma()
    
    # Calculate desired position
    new_x = gamma * agent1_pos[0] + (1 - gamma) * agent2_pos[0]
    new_y = gamma * agent1_pos[1] + (1 - gamma) * agent2_pos[1]
    
    # Calculate distance to human
    dist_to_human = distance([new_x, new_y], agent2_pos)
    
    # If too far from human, adjust position
    if dist_to_human > MAX_DISTANCE_FROM_HUMAN:
        # Calculate direction vector from human to desired position
        dx = new_x - agent2_pos[0]
        dy = new_y - agent2_pos[1]
        # Normalize and scale to maximum allowed distance
        length = math.sqrt(dx**2 + dy**2)
        if length > 0:
            dx = dx / length * MAX_DISTANCE_FROM_HUMAN
            dy = dy / length * MAX_DISTANCE_FROM_HUMAN
            new_x = agent2_pos[0] + dx
            new_y = agent2_pos[1] + dy
    
    # Smooth movement
    move_speed = 0.15  # Adjust this value to change how quickly X moves to its new position
    agent3_pos[0] += (new_x - agent3_pos[0]) * move_speed
    agent3_pos[1] += (new_y - agent3_pos[1]) * move_speed
    
    # Update direction
    dx = new_x - agent3_pos[0]
    dy = new_y - agent3_pos[1]
    agent3_dir = normalize_vector(dx, dy)
    
    return gamma

def reset():
    global agent1_pos, agent2_pos, agent3_pos, reached_goal
    global agent1_dir, agent2_dir, agent3_dir, current_target_idx, prev_human_pos
    
    agent1_pos = [300, 300]
    agent2_pos = [300, 300]
    agent3_pos = [300, 300]
    agent1_dir = [0, 0]
    agent2_dir = [0, 0]
    agent3_dir = [0, 0]
    reached_goal = False
    prev_human_pos = agent2_pos.copy()
    current_target_idx = 0
    
    generate_obstacles()  # Generate new obstacles first
    generate_targets()    # Then generate targets that avoid obstacles
    
    # Clear any pending auto-reset events
    pygame.time.set_timer(pygame.USEREVENT, 0)


def render(gamma):
    screen.fill(WHITE)
    

    for obstacle_pos in obstacles:
        pygame.draw.circle(screen, GRAY, (int(obstacle_pos[0]), int(obstacle_pos[1])), OBSTACLE_RADIUS)
    

    # Draw predicted path for green dot
    current_target = targets[current_target_idx]
    path_points = calculate_path_points(agent1_pos, current_target)
    draw_predicted_path(screen, path_points, (*GREEN, 128))  # Semi-transparent green
    
    # Draw all targets with numbers
    for i, target in enumerate(targets):
        pygame.draw.circle(screen, YELLOW, (int(target[0]), int(target[1])), 10)
        num_text = font.render(str(i+1), True, BLACK)
        screen.blit(num_text, (target[0] - 5, target[1] - 12))
    
    # Highlight current target with a ring
    pygame.draw.circle(screen, BLACK, (int(current_target[0]), int(current_target[1])), 12, 2)
    
    # Draw agents and their direction arrows
    pygame.draw.circle(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), 10)
    pygame.draw.circle(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), 10)
    pygame.draw.circle(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), 10)
    
    draw_arrow(screen, GREEN, (int(agent1_pos[0]), int(agent1_pos[1])), agent1_dir)
    draw_arrow(screen, BLUE, (int(agent2_pos[0]), int(agent2_pos[1])), agent2_dir)
    draw_arrow(screen, RED, (int(agent3_pos[0]), int(agent3_pos[1])), agent3_dir)

    # Draw labels
    screen.blit(font.render("W", True, BLACK), (agent1_pos[0] - 5, agent1_pos[1] - 12))
    screen.blit(font.render("H", True, BLACK), (agent2_pos[0] - 5, agent2_pos[1] - 12))
    screen.blit(font.render("X", True, BLACK), (agent3_pos[0] - 5, agent3_pos[1] - 12))

    # Display information
    gamma_text = font.render(f"Gamma: {gamma:.2f}", True, FONT_COLOR)
    screen.blit(gamma_text, (10, 10))
    formula_text = font.render(f"X = {gamma:.2f}W + {1-gamma:.2f}H", True, FONT_COLOR)
    screen.blit(formula_text, (10, 40))
    target_text = font.render(f"Current Target: {current_target_idx + 1}", True, FONT_COLOR)
    screen.blit(target_text, (10, 70))

    if reached_goal:
        reset_text = font.render("Goal Reached! Auto-resetting...", True, FONT_COLOR)
        screen.blit(reset_text, (150, 100))

    pygame.display.update()

# Main game loop
running = True
clock = pygame.time.Clock()
gamma = 0.5

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                reset()
        if event.type == pygame.USEREVENT:
            reset()  # Auto-reset when timer expires


    if not reached_goal:
        keys = pygame.key.get_pressed()
        dx, dy = 0, 0
        if keys[pygame.K_LEFT]:
            dx -= step_size
        if keys[pygame.K_RIGHT]:
            dx += step_size
        if keys[pygame.K_UP]:
            dy -= step_size
        if keys[pygame.K_DOWN]:
            dy += step_size

        if dx != 0 or dy != 0:
            move_agent2(dx, dy)
            current_target_idx = predict_human_target()
            move_agent1()
            gamma = update_agent3()

    render(gamma)
    clock.tick(30)

pygame.quit()